In [1]:
from nbodykit.source.catalog import CSVCatalog
from nbodykit.algorithms.paircount_tpcf.tpcf import SimulationBox2PCF
from nbodykit.cosmology.correlation import CorrelationFunction
from nbodykit.cosmology.power.halofit import HalofitPower
from nbodykit.cosmology import Cosmology
import emcee as mc




In [ ]:
def compute_xi_from_box(bins, boxdir):
    names  = ['x', 'y','z','vx','vy','vz']
    f = CSVCatalog(path.join(boxdir, 'uatu_z0p000.0'), names)

    downsample_idxs = np.random.choice(f['x'].shape[0], int(1e-1*f['x'].shape[0]), replace = False)
    d_idxs = np.zeros((f['x'].shape[0],))
    d_idxs[downsample_idxs] = 1
    f = f[d_idxs.astype(bool)]
    f['Position'] = np.c_[f['x'], f['y'], f['z']]
    s = SimulationBox2PCF('1d', f, bins, BoxSize = 512.0, show_progress = True)

    return np.array([x[0] for x in s.corr.data])

In [ ]:
def lnliklihood(theta, bins, obs_xi, invcov):

    Om, s8 = theta
    Ob = 0.022/(0.7**2)
    Ocdm = Om - Ob
    cosmo = Cosmology(h=0.7, T0_cmb=2.726,
                      Omega_b = 0.022,
                      Omega0_cdm=Ocdm,
                      P_k_max = 100.0,
                      n_s=0.96).match(s8)
    bc = (bins[1:] + bins[:-1])/2.0
    predicted_xi = CorrelationFunction(HalofitPower(cosmo, 0.0))(bc)
    delta =  predicted_xi - obs_xi
    return -0.5*np.sum(np.dot(delta, np.dot(invcov, delta)))
